### Converting preprocessed text into vectors using fasttext embedding.

In [3]:
from google.colab import drive
import numpy as np
import pandas as pd
drive.mount('/content/gdrive')
bengali_news_after_preprocessing = pd.read_pickle('/content/gdrive/My Drive/Projects/Bengali Text Classification/Bengali_Text_after_preprocessing.pkl')
from sklearn.externals import joblib
filename = '/content/gdrive/My Drive/Projects/Bengali Text Classification/fastText_Bangla_content_full.sav'
loaded_model = joblib.load(filename)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


/usr/local/lib/python3.6/dist-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [0]:
import keras.backend as K
import numpy as np
number_of_sample, max_number_of_words, word_vector_size = 40000, 100, 32
temp = bengali_news_after_preprocessing.loc[:number_of_sample-1,:max_number_of_words-1]

### Alternate method to get weird text 

In [53]:
#ord('ী') #2496
#ord('ু') #2497
c = chr(2496)+chr(2497)
c

'ীু'

In [0]:
temp = temp.replace(['ঘস', 'ফগ', 'ঝবঃ', 'ঋন', 'ঊঘ', '\u09e4', 'ওৎ', 'গথ', 'খঢ', 'ঝ’', 'ং', 'ঔ', 'ডড', 'গঘ','ঐব','ওঃ’ং','ুী','খষ','ি'], None)
X = np.zeros((number_of_sample, max_number_of_words, word_vector_size), dtype=K.floatx())
for i in temp.index:
  X[i,:,:] = loaded_model.wv[temp.loc[i,:]]

### preparing labels from csv

In [0]:
bengali_news = pd.read_pickle('/content/gdrive/My Drive/Projects/Bengali Text Classification/40k_bangla_newspaper_article.p')
bengali_news_dataframe = pd.DataFrame(bengali_news)
y = bengali_news_dataframe['category']

In [0]:
from sklearn import preprocessing
import keras
import numpy as np
le = preprocessing.LabelEncoder()
le.fit(y)
enc = le.transform(y)
y = keras.utils.to_categorical(enc)

### Train, Validation and Test Split

In [0]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X, y, shuffle = True, test_size=0.125)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, shuffle = True, test_size=0.20)

### Training CNN

In [0]:
from keras.models import Sequential
from keras.layers import Conv1D, Dropout, Dense, Flatten, LSTM, MaxPooling1D, Bidirectional
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, TensorBoard


model = Sequential()

model.add(Conv1D(32, kernel_size=3, activation='relu', padding='same',
                 input_shape=(max_number_of_words, word_vector_size)))
model.add(Conv1D(32, kernel_size=3, activation='relu', padding='same'))
model.add(Conv1D(32, kernel_size=3, activation='relu', padding='same'))
model.add(MaxPooling1D(pool_size=3))
model.add(Flatten())
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.2))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.25))
model.add(Dense(256, activation='sigmoid'))
model.add(Dropout(0.25))

model.add(Dense(13, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.0001, decay=1e-6), metrics=['accuracy'])

In [60]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 100, 32)           3104      
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 100, 32)           3104      
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 100, 32)           3104      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 33, 32)            0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1056)              0         
_________________________________________________________________
dense_5 (Dense)              (None, 256)               270592    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
__________

In [61]:
history = model.fit(X_train, y_train, batch_size= 500, shuffle=True, epochs= 100, validation_data=(X_val, y_val))

Train on 35000 samples, validate on 4000 samples
Epoch 1/100
35000/35000 [==============================] - 2s 69us/step - loss: 2.1549 - acc: 0.2504 - val_loss: 1.9087 - val_acc: 0.3150
Epoch 2/100
35000/35000 [==============================] - 1s 36us/step - loss: 1.9307 - acc: 0.3135 - val_loss: 1.7530 - val_acc: 0.4253
Epoch 3/100
35000/35000 [==============================] - 1s 37us/step - loss: 1.6954 - acc: 0.4415 - val_loss: 1.4604 - val_acc: 0.5525
Epoch 4/100
35000/35000 [==============================] - 1s 37us/step - loss: 1.4199 - acc: 0.5769 - val_loss: 1.2266 - val_acc: 0.6448
Epoch 5/100
35000/35000 [==============================] - 1s 36us/step - loss: 1.2339 - acc: 0.6359 - val_loss: 1.1050 - val_acc: 0.6573
Epoch 6/100
35000/35000 [==============================] - 1s 36us/step - loss: 1.1409 - acc: 0.6533 - val_loss: 1.0346 - val_acc: 0.6743
Epoch 7/100
35000/35000 [==============================] - 1s 36us/step - loss: 1.0754 - acc: 0.6705 - val_loss: 0.9858 - v

In [0]:
predicts = model.predict(X_test)
import numpy as np
def decode(le, one_hot):
    dec = np.argmax(one_hot, axis=1)
    return le.inverse_transform(dec)
y_test = decode(le, y_test)
y_preds = decode(le, predicts)

In [63]:
from sklearn import metrics

print(metrics.accuracy_score(y_test, y_preds))

print(metrics.confusion_matrix(y_test, y_preds))

print(metrics.classification_report(y_test, y_preds))

0.802
[[  0   2   0   0   0   0   0   5   0   2   0   0]
 [  0 251   0   9   2   1   4   3   0  14   0   2]
 [  0   5   0   0   0   2   0   0   0   0   0   0]
 [  0  13   0 115   1   0   0   0   0   7   0   4]
 [  0   3   0   0   7   0   0   3   0   2   0   0]
 [  0   2   0   0   0  53   0   1   0   0   0   1]
 [  0   6   0   1   0   1  31   0   0   9   0   0]
 [  0   2   0   0   2   1   0  19   0   5   1   2]
 [  0   2   0   0   0   1   2   0   0   1   1   1]
 [  3  28   0  12   3   1   3   7   0 216   0   4]
 [  0   0   0   0   0   3   0   0   0   0  79   0]
 [  0   0   0   3   2   1   0   1   0   1   0  31]]
                    precision    recall  f1-score   support

art-and-literature       0.00      0.00      0.00         9
        bangladesh       0.80      0.88      0.84       286
       durporobash       0.00      0.00      0.00         7
           economy       0.82      0.82      0.82       140
         education       0.41      0.47      0.44        15
     entertainment  

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [0]:
accuracy, val_accuracy = np.array(history.history["acc"]), np.array(history.history["val_acc"])
accuracy, val_accuracy = accuracy.reshape(100,1), val_accuracy.reshape(100,1)
accuracies = np.concatenate((accuracy,val_accuracy),axis=1)
np.savetxt('/content/gdrive/My Drive/Projects/Bengali Text Classification/temp.csv',accuracies,delimiter=",")